In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import time as time
from datetime import datetime
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import pickle

In [2]:
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
player_data = []

for letter in letters:
    url = f"https://www.footballdb.com/players/current.html?letter={letter}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        player_rows = soup.find_all('div', class_='tr')
        for row in player_rows:
            player_name_tag = row.find('div', class_='td').find('a', href=True)
            if player_name_tag:
                player_name = player_name_tag.get_text(strip=True)
            position_tag = row.find_all('div', class_='td')[1]
            player_position = position_tag.get_text(strip=True) if position_tag else 'N/A'
            # team_tag = row.find_all('div', class_='td')[2]
            # player_team = team_tag.get_text(strip=True) if team_tag else 'N/A'
            # secondary_team_tag = row.find_all('div', class_='td')[3]
            # secondary_team = secondary_team_tag.get_text(strip=True) if secondary_team_tag else 'N/A'
            player_data.append([player_name, player_position])

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
df_names = pd.DataFrame(player_data, columns=["Player Name", "Position"])
# df.to_csv("players_data.csv", index=False, encoding="utf-8")
# print(f"Saved {len(df)} player records to players_data.csv")
print(df_names)
names_list = df_names['Player Name'].tolist()
    

             Player Name Position
0     Abanikanda, Israel       RB
1        Abdullah, Ameer       RB
2        Abdullah, Yasir       LB
3       Abernathy, Micah       DB
4         Abraham, Micah       DB
...                  ...      ...
2553      Zeitler, Kevin       OG
2554      Zierer, Kilian       OT
2555         Zinter, Zak       OG
2556      Zuerlein, Greg        K
2557      Zylstra, Shane       TE

[2558 rows x 2 columns]


In [3]:
# Cell takes about ten minutes to run
nfl_teams = [
    "arizona-cardinals", "atlanta-falcons", "baltimore-ravens", "buffalo-bills",
    "carolina-panthers", "chicago-bears", "cincinnati-bengals", "cleveland-browns",
    "dallas-cowboys", "denver-broncos", "detroit-lions", "green-bay-packers",
    "houston-texans", "indianapolis-colts", "jacksonville-jaguars", "kansas-city-chiefs",
    "los-angeles-chargers", "los-angeles-rams", "miami-dolphins", "minnesota-vikings",
    "new-england-patriots", "new-orleans-saints", "new-york-giants", "new-york-jets",
    "philadelphia-eagles", "pittsburgh-steelers", "san-francisco-49ers", "seattle-seahawks",
    "tampa-bay-buccaneers", "tennessee-titans", "washington-commanders"
]

counter = 0
player_data = []
D = 2005
print(D)
for yearo in range(D, 2024):
    year = 4026 - yearo
    time.sleep(10) # Add some sleep time to not overload request limit
    for team in nfl_teams: 
        url = f"https://www.footballdb.com/teams/nfl/{team}/roster/{year}"
        if team == "washington-commanders" and year >= 1936 and year < 2020:
            url = f"https://www.footballdb.com/teams/nfl/washington-redskins/roster/{year}"
        elif team == "washington-commanders" and (year == 2021 or year == 2020):
            url = f"https://www.footballdb.com/teams/nfl/washington-football-team/roster/{year}"
        elif team == "los-angeles-rams" and year >= 1995 and year <= 2015:
            url = f"https://www.footballdb.com/teams/nfl/st-louis-rams/roster/{year}"
        elif team == "los-angeles-chargers" and year <= 2016:
            url = f"https://www.footballdb.com/teams/nfl/san-diego-chargers/roster/{year}"
        elif team == "tennessee-titans" and year <= 1998:
            url = f"https://www.footballdb.com/teams/nfl/tennessee-oilers/roster/{year}"    
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            print(f"Got data for the {team} in year {year}")
            soup = BeautifulSoup(response.text, 'html.parser')
            roster_table = soup.find('div', class_='divtable divtable-striped divtable-mobile')
            player_rows = roster_table.find_all('div', class_='tr')
            
            for row in player_rows:
                # number_tag = row.find('div', class_='td w10 m20 rostercell_num')
                # player_number = number_tag.get_text(strip=True) if number_tag else 'N/A'
                name_tag = row.find('div', class_='td w20 m80 rostercell_name')
                player_name_tag = name_tag.find('a', href=True) if name_tag else None
                player_name = player_name_tag.get_text(strip=True) if player_name_tag else 'N/A'
                position_tag = row.find('div', class_='td w10 rostercell_pos hidden-xs')
                player_position = position_tag.get_text(strip=True) if position_tag else 'N/A'
                # height_tag = row.find('div', class_='td w10 rostercell_ht hidden-xs')
                # player_height = height_tag.get_text(strip=True) if height_tag else 'N/A'
                # weight_tag = row.find('div', class_='td w10 rostercell_wt hidden-xs')
                # player_weight = weight_tag.get_text(strip=True) if weight_tag else 'N/A'
                # age_tag = row.find('div', class_='td w10 rostercell_age hidden-xs')
                # player_age = age_tag.get_text(strip=True) if age_tag else 'N/A'
                # college_tag = row.find('div', class_='td w30 rostercell_coll hidden-xs')
                # player_college = college_tag.get_text(strip=True) if college_tag else 'N/A'
                player_data.append([team, year, player_name, position_tag])

        else:
            print(f"FAILED to retrieve the page for team {team} in year {year}. Status code: {response.status_code}")
df = pd.DataFrame(player_data, columns=["Team", "Year", "Player Name", "Player Position"])
df.to_csv(f"team_composition_{D}_to_2023.csv", index=False, encoding="utf-8")
print(df.shape)

2005


KeyboardInterrupt: 

In [3]:
df.to_csv(f"team_composition_{D}_to_2023.csv", index=False, encoding="utf-8")
print(df)
nfl_team_compositions = pd.read_csv(f'team_composition_{D}_to_2023.csv')
print(nfl_team_compositions)
nfl_team_compositions['Team'] = nfl_team_compositions['Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper())
print(nfl_team_compositions)

NameError: name 'df' is not defined

In [4]:
def get_nfl_year(date_str):
    game_date = datetime.strptime(date_str, "%Y-%m-%d")
    if game_date.month in [1, 2]:  # For January and February, it's the previous NFL season
        print(game_date.year - 1)
        return game_date.year - 1
    else:
        print(game_date.year)
        return game_date.year

In [5]:
df = pd.read_csv('nfl-game-info.csv')
df['Date'] = df['Date'].apply(get_nfl_year)

def determine_winner_loser(row):
    if row['Home Team'] == "washington-redskins" and row['Date'] >= 1936 and row['Date'] < 2020:
        row['Home Team'] = "washington-commanders"
    elif row['Home Team'] == "washington-football-team" and (row['Date'] == 2021 or row['Date'] == 2020):
        row['Home Team'] = "washington-commanders"
    elif row['Away Team'] == "washington-redskins" and row['Date'] >= 1936 and row['Date'] < 2020:
        row['Away Team'] = "washington-commanders"
    elif row['Away Team'] == "washington-football-team" and (row['Date'] == 2021 or row['Date'] == 2020):
        row['Away Team'] = "washington-commanders"
    
    if row['Home Team'] == "st-louis-rams" and row['Date'] >= 1995 and row['Date'] <= 2015:
        row['Home Team'] = "los-angeles-rams"
    elif row['Away Team'] == "st-louis-rams" and row['Date'] >= 1995 and row['Date'] <= 2015:
        row['Away Team'] = "los-angeles-rams"
    
    if row['Away Team'] == "san-diego-chargers" and row['Date'] <= 2016:
        row['Away Team'] = "los-angeles-chargers"
    elif row['Home Team'] == "san-diego-chargers" and row['Date'] <= 2016:
        row['Home Team'] = "los-angeles-chargers"

    if row['Home Team'] == "tennessee-oilers" and row['Date'] <= 1998:
        row['Home Team'] = "tennessee-titans"
    elif row['Away Team'] == "tennessee-oilers" and row['Date'] <= 1998:
        row['Away Team'] = "tennessee-titans"

    # Now compare the scores to determine the winner and loser
    if row['Home Score'] > row['Away Score']:
        return pd.Series([row['Home Team'], row['Away Team']])
    elif row['Home Score'] < row['Away Score']:
        return pd.Series([row['Away Team'], row['Home Team']])
    else:
        return pd.Series([None, None])

# Apply the function to determine winners and losers, and handle exceptions
df[['Winning Team', 'Losing Team']] = df.apply(determine_winner_loser, axis=1)
df['Winning Team'] = df['Winning Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper() if pd.notnull(x) else x)
df['Losing Team'] = df['Losing Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper() if pd.notnull(x) else x)
df.to_csv('nfl-game-info-updated.csv', index=False)
print(df.head())



2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2024
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023
2023


In [6]:
D = 2005
nfl_game_info = pd.read_csv('nfl-game-info-updated.csv')
nfl_team_compositions = pd.read_csv(f'team_composition_{D}_to_2023.csv')
nfl_team_compositions['Team'] = nfl_team_compositions['Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper())
team_to_players = defaultdict(lambda: defaultdict(list))

for _, row in nfl_team_compositions.iterrows():
    team = row['Team']
    year = row['Year']
    player = row['Player Name']
    team_to_players[team][year].append(player)
adj_matrix = defaultdict(lambda: defaultdict(int))
for _, game in nfl_game_info.iterrows():
    winner = game['Winning Team']
    loser = game['Losing Team']
    nfl_year = game['Date']
    
    if winner in team_to_players and nfl_year in team_to_players[winner]:
        winning_players = team_to_players[winner][nfl_year]
    else:
        continue

    if loser in team_to_players and nfl_year in team_to_players[loser]:
        losing_players = team_to_players[loser][nfl_year]
    else:
        continue
    for winner_player in winning_players:
        for loser_player in losing_players:
            adj_matrix[winner_player][loser_player] += 1

adj_matrix_df = pd.DataFrame.from_dict({player: dict(edges) for player, edges in adj_matrix.items()}).fillna(0)
print(adj_matrix_df)
adj_matrix_df.to_csv('player_adjacency_matrix.csv')


                  Cam Akers  Landen Akers  Brian Allen  Tremayne Anchrum  \
Hakeem Adeniji          1.0           1.0          5.0               1.0   
Brandon Allen           1.0           1.0          5.0               1.0   
Ricardo Allen           1.0           1.0          5.0               1.0   
Eli Apple               1.0           1.0          5.0               1.0   
Chidobe Awuzie          2.0           1.0          6.0               2.0   
...                     ...           ...          ...               ...   
Troy Bienemann          0.0           0.0          0.0               0.0   
David Holloway          0.0           0.0          0.0               0.0   
Bhawoh Jue              0.0           0.0          0.0               0.0   
Mel Mitchell            0.0           0.0          0.0               0.0   
Marcellus Rivers        0.0           0.0          0.0               0.0   

                  Chatarius Atwell  Odell Beckham Jr.  Kendall Blanton  \
Hakeem Adenij

In [7]:
D = 2005
nfl_game_info = pd.read_csv('nfl-game-info-updated.csv')
nfl_team_compositions = pd.read_csv(f'team_composition_{D}_to_2023.csv')
nfl_team_compositions['Team'] = nfl_team_compositions['Team'].apply(lambda x: x.replace(" ", "").replace("-", "").upper())

team_to_players = defaultdict(lambda: defaultdict(list))

for _, row in nfl_team_compositions.iterrows():
    team = row['Team']
    year = row['Year']
    player = row['Player Name']
    team_to_players[team][year].append(player)

all_players = set()
for team in team_to_players.values():
    for players in team.values():
        all_players.update(players)

player_encoder = {player: idx for idx, player in enumerate(sorted(all_players))} # USE IN FUTURE PARTS
adj_matrix = defaultdict(lambda: defaultdict(int))

for _, game in nfl_game_info.iterrows():
    winner = game['Winning Team']
    loser = game['Losing Team']
    nfl_year = game['Date']
    
    if winner in team_to_players and nfl_year in team_to_players[winner]:
        winning_players = team_to_players[winner][nfl_year]
    else:
        continue

    if loser in team_to_players and nfl_year in team_to_players[loser]:
        losing_players = team_to_players[loser][nfl_year]
    else:
        continue

    for winner_player in winning_players:
        winner_idx = player_encoder[winner_player]
        for loser_player in losing_players:
            loser_idx = player_encoder[loser_player]
            adj_matrix[winner_idx][loser_idx] += 1

adj_matrix_df = pd.DataFrame.from_dict({player: dict(edges) for player, edges in adj_matrix.items()}).fillna(0)
print(adj_matrix_df)

adj_matrix_df.to_csv('player_adjacency_matrix.csv', index=False)

import pickle
with open('player_encoder.pkl', 'wb') as f:
    pickle.dump(player_encoder, f)

      1253  5646  1038  8678  1486  6835  5237  468   790   8345  ...  6157  \
3494   1.0   1.0   5.0   1.0   1.0   5.0   1.0   2.0   1.0   1.0  ...   0.0   
871    1.0   1.0   5.0   1.0   1.0   5.0   1.0   2.0   1.0   1.0  ...   0.0   
7334   1.0   1.0   5.0   1.0   1.0   4.0   2.0   1.0   1.0   1.0  ...   0.0   
3038   1.0   1.0   5.0   1.0   1.0   3.0   2.0   1.0   1.0   1.0  ...   0.0   
1505   2.0   1.0   6.0   2.0   1.0   4.0   1.0   2.0   1.0   2.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
8757   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2359   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
732    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6313   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5942   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      7341  3438  8602  2864  3176  7287  18    238

In [8]:
adj_matrix_np = adj_matrix_df.to_numpy()
total_elements = adj_matrix_np.size
print(total_elements)
zero_elements = np.sum(adj_matrix_np == 0)
print(zero_elements)
sparsity = zero_elements / total_elements

print(f"Sparsity with naive representation: {sparsity:.4f}")

import scipy.sparse as sp

adj_matrix_sparse = sp.csr_matrix(adj_matrix_df.to_numpy())
sparsity_sparse = 1 - (adj_matrix_sparse.nnz / adj_matrix_sparse.size)
print(adj_matrix_sparse)
print(f"Sparsity with sparse matrix: {sparsity_sparse:.4f}")
sp.save_npz('player_adjacency_matrix_sparse.npz', adj_matrix_sparse)

61583146
51472838
Sparsity with naive representation: 0.8358
  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	5.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	5.0
  (0, 6)	1.0
  (0, 7)	2.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  :	:
  (7857, 6438)	1.0
  (7857, 6530)	1.0
  (7857, 6709)	1.0
  (7857, 6775)	1.0
  (7857, 6776)	1.0
  (7857, 6778)	1.0
  (7857, 6779)	1.0
  (7857, 6781)	1.0
  (7857, 6794)	1.0
  (7857, 6996)	1.0
  (7857, 6997)	1.0
  (7857, 6998)	1.0
  (7857, 7000)	1.0
  (7857, 7002)	1.0
  (7857, 7004)	1.0
  (7857, 7006)	1.0
  (7857, 7155)	1.0
  (7857, 7156)	1.0
  (7857, 7157)	1.0
  (7857, 7158)	1.0
  (7857, 7159)	1.0
  (7857, 7160)	1.0
  (7857, 7161)	1.0
  (7857, 7162)	1.0
  (7857, 7163)	1.0
Sparsity with sparse matrix: 0.0000


In [9]:
G = nx.DiGraph()
for winner_idx, losers in adj_matrix.items():
    for loser_idx, count in losers.items():
        G.add_edge(winner_idx, loser_idx, weight=count)

with open('player_encoder.pkl', 'wb') as f:
    pickle.dump(player_encoder, f)

# Optionally: Save the graph to a file for future use
# nx.write_graphml(G, 'player_interaction_graph.graphml')

adj_matrix_df = pd.DataFrame.from_dict({player_idx: dict(edges) for player_idx, edges in adj_matrix.items()}).fillna(0)
adj_matrix_df.to_csv('player_adjacency_matrix.csv', index=False)
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

Graph has 7858 nodes and 10110308 edges.


In [10]:
nfl_game_info = pd.read_csv('nfl-game-info-updated.csv')

# Get the list of years from the 'Date' column (assuming it's in 'YYYY-MM-DD' format)
# nfl_game_info['Year'] = pd.to_datetime(nfl_game_info['Date']).dt.year

# Choose 3 years as the test years (e.g., 2021, 2022, 2023) and the rest as train years
test_years = [2021, 2022, 2023]
train_years = [year for year in nfl_game_info['Date'].unique() if year not in test_years]

# Split the dataset into train and test
train_games = nfl_game_info[nfl_game_info['Date'].isin(train_years)]
test_games = nfl_game_info[nfl_game_info['Date'].isin(test_years)]

print(f"Training years: {train_years}")
print(f"Test years: {test_years}")

Training years: [2024, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006]
Test years: [2021, 2022, 2023]


In [11]:
node_degrees = dict(G.degree())  # In-degree + out-degree
print("Finally finished degrees")
node_pagerank = nx.pagerank(G)  # PageRank centrality
print("Finally finished pagerank")

# Add graph features to the player_encoder mapping (for use in the model later)
graph_features = {}
for player, player_idx in player_encoder.items():
    graph_features[player_idx] = {
        "degree": node_degrees.get(player_idx, 0),
        "pagerank": node_pagerank.get(player_idx, 0)
    }

# Optionally: Create a DataFrame to represent the features for each player
graph_features_df = pd.DataFrame.from_dict(graph_features, orient='index')
print(graph_features_df.head())

Finally finished degrees
Finally finished pagerank
   degree  pagerank
0    4027  0.000184
1    5200  0.000292
2    2740  0.000107
3    4479  0.000321
4    1473  0.000064


In [12]:
def get_team_features(game, team_to_players, player_encoder, graph_features):
    """
    Get the features for a team based on the players in the team.
    The features are averaged for the players in the team.
    """
    home_team = game['Home Team']
    away_team = game['Away Team']
    year = pd.to_datetime(game['Date']).year
    
    # Get the players for each team in the game
    home_players = team_to_players[home_team].get(year, [])
    away_players = team_to_players[away_team].get(year, [])
    
    # If a team has no players in the given year, return empty arrays
    if not home_players or not away_players:
        return np.array([]), np.array([])

    # Get indices for each player's features
    home_indices = [player_encoder[player] for player in home_players if player in player_encoder]
    away_indices = [player_encoder[player] for player in away_players if player in player_encoder]
    
    # If there are no players in the player_encoder, return empty arrays
    if not home_indices or not away_indices:
        return np.array([]), np.array([])
    
    # Get the features for these players from the graph_features
    home_features = graph_features[home_indices].mean(axis=0)  # Average of the players' features
    away_features = graph_features[away_indices].mean(axis=0)  # Average of the players' features
    
    return home_features, away_features

# Example usage:
train_features = []
train_labels = []

# Assuming `train_games` contains the games data
for game in train_games.iterrows():
    game = game[1]  # Get the game row
    
    home_features, away_features = get_team_features(game, team_to_players, player_encoder, graph_features)
    
    # If the features are empty, skip this game
    if home_features.size == 0 or away_features.size == 0:
        continue
    
    # Combine home and away team features (e.g., concatenate them)
    game_features = np.concatenate([home_features, away_features])
    train_features.append(game_features)
    
    # Label: 1 if home team wins, 0 if away team wins
    label = 1 if game['Winning Team'] == home_team else 0
    train_labels.append(label)

# Convert to numpy arrays
train_features = np.array(train_features)
train_labels = np.array(train_labels)


In [13]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        
        # Define layers
        self.conv1 = GCNConv(input_dim, hidden_dim)  # First GCN layer
        self.conv2 = GCNConv(hidden_dim, output_dim)  # Second GCN layer

    def forward(self, data):
        # Apply the first GCN layer and activation
        x = F.relu(self.conv1(data.x, data.edge_index))
        
        # Apply the second GCN layer
        x = self.conv2(x, data.edge_index)
        
        return x

# Define model parameters
input_dim = 64  # Number of input features for each player (e.g., stats or embeddings)
hidden_dim = 128  # Hidden layer size
output_dim = 1  # Output for each player (or team), could be win/loss prediction

# Instantiate the model
model = GNNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
print(model)

GNNModel(
  (conv1): GCNConv(64, 128)
  (conv2): GCNConv(128, 1)
)


In [14]:
import torch
from torch_geometric.data import Data
import torch_geometric.utils as pyg_utils
import networkx as nx
import numpy as np

# Assuming you already have the adjacency matrix and player encoder
# Convert the adjacency matrix to an edge index (required by PyTorch Geometric)

# Convert NetworkX graph G to PyTorch Geometric format
edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()  # Edge list as tensor (2, num_edges)
edge_attr = torch.tensor([G[u][v]['weight'] for u, v in G.edges], dtype=torch.float32)  # Edge weights (number of times player i beat player j)

# Define node features for players. This is just an example.
# If you have player statistics, you can use that instead.
num_players = len(player_encoder)
node_features = torch.randn(num_players, 64)  # Random features for now (64-dimensional)

# Create the data object for PyTorch Geometric
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)

# Now, you can use `data` in your model.


In [ ]:
import torch.optim as optim
import tqdm

# Create the game labels (1 = home team wins, 0 = away team wins)
# These labels will be created based on the `train_games` and `test_games` data
def create_game_labels(game_df, team_to_players, player_encoder):
    labels = []
    for _, game in tqdm.tqdm(game_df.iterrows(), total=game_df.shape[0], desc="Creating game labels"):
        home_team = game['Home Team']
        away_team = game['Away Team']
        year = pd.to_datetime(game['Date']).year
        
        # Get the players for each team in the game
        home_players = team_to_players[home_team][year]
        away_players = team_to_players[away_team][year]
        
        # Combine home and away players
        home_idx = [player_encoder[player] for player in home_players]
        away_idx = [player_encoder[player] for player in away_players]
        
        # Aggregate the game result
        label = 1 if game['Winning Team'] == home_team else 0
        labels.append((home_idx, away_idx, label))
    return labels

# Create labels for the training data
train_labels = create_game_labels(train_games, team_to_players, player_encoder)
test_labels = create_game_labels(test_games, team_to_players, player_encoder)

# Convert to tensors
train_labels_tensor = torch.tensor([label[2] for label in train_labels], dtype=torch.float32)
test_labels_tensor = torch.tensor([label[2] for label in test_labels], dtype=torch.float32)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Training loop with tqdm for progress tracking
num_epochs = 10
for epoch in tqdm.tqdm(range(num_epochs), desc="Training Epochs", ncols=100, position=0):
    model.train()
    optimizer.zero_grad()

    # Forward pass using the 'data' object
    output = model(data)  # This is the output for all players (nodes)
    
    # For now, let's use a simple way to combine team features (e.g., take mean)
    home_team_output = output[train_labels_tensor[:, 0].long()].mean(dim=0)
    away_team_output = output[train_labels_tensor[:, 1].long()].mean(dim=0)
    
    # Compute prediction for the game
    pred = torch.sigmoid(home_team_output - away_team_output)
    
    # Compute the loss
    loss = loss_fn(pred, train_labels_tensor)
    
    # Backward pass
    loss.backward()
    optimizer.step()

    # Optionally, print loss at every epoch
    tqdm.tqdm.write(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation loop with tqdm for progress tracking
model.eval()
test_preds = []
test_labels_list = []
with torch.no_grad():
    for i in tqdm.tqdm(range(len(test_labels_tensor)), desc="Evaluating", ncols=100, position=1):
        home_team_output = output[test_labels_tensor[i, 0].long()].mean(dim=0)
        away_team_output = output[test_labels_tensor[i, 1].long()].mean(dim=0)
        
        # Compute prediction for the game
        pred = torch.sigmoid(home_team_output - away_team_output)
        test_preds.append(pred.item())
        test_labels_list.append(test_labels_tensor[i].item())

# Convert lists to tensors for loss calculation
test_preds_tensor = torch.tensor(test_preds, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels_list, dtype=torch.float32)

# Compute the test loss
test_loss = loss_fn(test_preds_tensor, test_labels_tensor)
print(f"Test Loss: {test_loss.item():.4f}")


Training Epochs:   0%|                                                       | 0/10 [00:00<?, ?it/s]